# 1. Acquiring Calibration Images (Aravis port)

Port of the original Lucid Arena SDK notebook to use
[Aravis](https://github.com/AravisProject/aravis), an open-source GigE Vision library.

Captures left/right image pairs of a checkerboard and saves them to
`stereoLeft/` and `stereoRight/` for use in `2.Calibration.ipynb`.

**Press `s`** to save a pair &nbsp;|&nbsp; **Press `q`** to quit.

## Prerequisites

### Hardware
- Lucid Phoenix PHD 1.6 MP Dual Extended-Head (IMX273)
- Connected via GigE (1000BASE-T); NIC configured for jumbo frames (MTU 9000) is recommended

### Software
- Aravis ≥ 0.8 with GObject Introspection support (`gir1.2-aravis-0.8`)
- `python3-gi` (PyGObject)
- `opencv-python`, `numpy`

On Ubuntu/Debian:
```bash
sudo apt install gir1.2-aravis-0.8 python3-gi
pip install opencv-python numpy
```

### Calibration target
- 9 × 6 inner-corner checkerboard with 25 mm squares (see `9x6_1-8cm_chessboard.png`)
- Minimum **30 image pairs** from varied angles (≤ 45°) covering the full frame

In [ ]:
!pip install opencv-python numpy

In [ ]:
import gi
gi.require_version('Aravis', '0.8')
from gi.repository import Aravis

import cv2
import numpy as np
import os
import time

## Configuration

In [ ]:
NUMBER_OF_IMAGES = 30  # recommended minimum

# IMX273 native resolution: 1440 × 1080 per sensor.
# With 2:1 sensor binning (2×2 average), each sensor outputs 720 × 540.
# In ImagerOutputSelector='All' mode the two binned sensors are interleaved
# column-by-column into a single frame: 2 × 720 = 1440 wide, 540 tall.
DUAL_WIDTH  = 1440
DUAL_HEIGHT = 540

In [ ]:
def adjust_gamma(image, gamma=1.0):
    """Apply gamma correction via a lookup table."""
    inv_gamma = 1.0 / gamma
    table = np.array(
        [((i / 255.0) ** inv_gamma) * 255 for i in range(256)]
    ).astype(np.uint8)
    return cv2.LUT(image, table)

## Device discovery

In [ ]:
Aravis.update_device_list()
n_devices = Aravis.get_n_devices()

if n_devices == 0:
    raise RuntimeError(
        "No GigE Vision camera found. "
        "Check network connection and firewall settings."
    )

print(f"Found {n_devices} device(s):")
for i in range(n_devices):
    print(f"  [{i}] {Aravis.get_device_id(i)}")

# Change the index if you have multiple cameras on the network.
camera = Aravis.Camera.new(Aravis.get_device_id(0))
device = camera.get_device()
print(f"\nConnected to: {camera.get_model_name()}")

## Camera configuration

The GenICam nodes (`ImagerOutputSelector`, `IspBayerPattern`, `PixelFormat`, …)
are Lucid-proprietary but are fully accessible via Aravis because the camera
exposes them in its standard GenICam XML device description.

In [ ]:
# Negotiate the largest supported GigE packet size with the NIC.
# Requires jumbo frames (MTU ≥ 9000) on the host NIC for best throughput.
camera.gv_auto_packet_size()

# Select combined dual-head output (both imagers in one frame).
device.set_string_feature_value("ImagerOutputSelector", "All")

# --- Sensor binning ----------------------------------------------------------
# Enable 2:1 sensor-level binning (average mode) on both axes.
# Sensor binning reduces data before readout, giving higher SNR and lower
# bandwidth than digital (post-readout) binning.
# Result: 1440 × 1080 → 720 × 540 per sensor.
device.set_string_feature_value("BinningSelector",      "Sensor")
device.set_integer_feature_value("BinningVertical",     2)
device.set_integer_feature_value("BinningHorizontal",   2)
device.set_string_feature_value("BinningVerticalMode",   "Average")
device.set_string_feature_value("BinningHorizontalMode", "Average")
print(f"Binning         : {device.get_integer_feature_value('BinningHorizontal')}×"
      f"{device.get_integer_feature_value('BinningVertical')} "
      f"({device.get_string_feature_value('BinningHorizontalMode')})")
# -----------------------------------------------------------------------------

# Set the combined frame dimensions (after binning).
device.set_integer_feature_value("Width",  DUAL_WIDTH)
device.set_integer_feature_value("Height", DUAL_HEIGHT)

# Detect mono vs colour and pick the appropriate dual pixel format.
isp_bayer_pattern = device.get_string_feature_value("IspBayerPattern")
if isp_bayer_pattern != "NONE":
    device.set_string_feature_value("PixelFormat", "DualBayerRG8")
else:
    device.set_string_feature_value("PixelFormat", "DualMono8")

print(f"IspBayerPattern : {isp_bayer_pattern}")
print(f"PixelFormat     : {device.get_string_feature_value('PixelFormat')}")
print(f"Frame size      : {device.get_integer_feature_value('Width')} × "
      f"{device.get_integer_feature_value('Height')}")

## Acquisition loop

Press **`s`** to save the current left/right pair.  
Press **`q`** to stop streaming and close windows.

In [ ]:
# Pre-allocate stream buffers.
# Aravis manages its own buffer queue; we push empty buffers that the
# camera fills and we later pop for processing.
payload = camera.get_payload()
stream  = camera.create_stream(None, None)
for _ in range(5):
    stream.push_buffer(Aravis.Buffer.new_allocate(payload))

# Create output directories.
cwd        = os.getcwd()
left_path  = os.path.join(cwd, "stereoLeft")
right_path = os.path.join(cwd, "stereoRight")
os.makedirs(left_path,  exist_ok=True)
os.makedirs(right_path, exist_ok=True)

cv2.namedWindow("Left",  cv2.WINDOW_NORMAL)
cv2.namedWindow("Right", cv2.WINDOW_NORMAL)

print("Streaming. Press 's' to save a pair, 'q' to quit.")
print(f"Target: {NUMBER_OF_IMAGES} image pairs")

camera.start_acquisition()
num = 0

try:
    while True:
        # Block up to 1 second for the next frame (timeout in microseconds).
        buf = stream.timeout_pop_buffer(1_000_000)

        if buf is None:
            print("Timeout waiting for frame — check network connection.")
            continue

        if buf.get_status() != Aravis.BufferStatus.SUCCESS:
            stream.push_buffer(buf)
            continue

        # Decode the interleaved dual-head frame.
        data   = buf.get_data()
        height = buf.get_image_height()
        width  = buf.get_image_width()

        # Wrap raw bytes in a numpy array (Mono8 / BayerRG8 → 1 byte/pixel).
        arr = np.frombuffer(data, dtype=np.uint8)[: height * width].reshape(height, width)

        # Split interleaved columns: even = left sensor, odd = right sensor.
        # After 2:1 binning each sub-image is 720 × 540 (one IMX273 sensor).
        img0 = arr[:, 0::2].copy()  # left
        img1 = arr[:, 1::2].copy()  # right

        # Re-queue the buffer before any slow operations.
        stream.push_buffer(buf)

        img0 = adjust_gamma(img0, 2.5)
        img1 = adjust_gamma(img1, 2.5)

        if isp_bayer_pattern != "NONE":
            img0 = cv2.cvtColor(img0, cv2.COLOR_BAYER_BG2BGR)
            img1 = cv2.cvtColor(img1, cv2.COLOR_BAYER_BG2BGR)

        cv2.imshow("Left",  img0)
        cv2.imshow("Right", img1)

        key = cv2.waitKey(1) & 0xFF

        if key == ord("q"):
            break
        elif key == ord("s"):
            cv2.imwrite(os.path.join(left_path,  f"imageL{num}.png"), img0)
            cv2.imwrite(os.path.join(right_path, f"imageR{num}.png"), img1)
            print(f"  Saved pair {num + 1} / {NUMBER_OF_IMAGES}")
            num += 1

finally:
    camera.stop_acquisition()
    cv2.destroyAllWindows()
    print(f"\nCaptured {num} image pairs.")
    print("Open 2.Calibration.ipynb to continue.")